In [ ]:
import os
import torch
import json
import pandas as pd
import seaborn as sns
import importlib
import image_net_python

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(0)
from utils import *

In [ ]:
importlib.reload(image_net_python)

In [ ]:
forward_normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
pretrained_model = models.resnet101(pretrained=True)
pretrained_model.to(device)
pretrained_model.eval()

lay2 = torch.nn.Softmax(dim=1)

CLASS_URL = 'https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl'
class_dict = pickle.load(urllib.request.urlopen(CLASS_URL))

print("done")

In [ ]:
def applyMask(new_w, batch_limits, mask, c_limits, sig_height, conv_size, precision_depth=2):
    
    lay = torch.nn.Conv1d(1,1,conv_size)

    #Manually setting the weights and bias so the  shutter acts as a box filter
    lay.weight.data = torch.full([1,1,conv_size,1], 1/conv_size, requires_grad=True, dtype=torch.float, device=device)
    lay.bias.data = torch.zeros(1, requires_grad=True, dtype=torch.float, device=device)
    
    sz = new_w.shape[1]
    batch = batch_limits[1]-batch_limits[0]          
    
    # EOT sampling for ambient light and shift
    c = torch.rand([batch,1,1,1], device=device) * (c_limits[1] - c_limits[0]) + c_limits[0]
    shift = torch.tensor(range(batch_limits[0],batch_limits[1]), dtype=torch.int)
    #shift = torch.from_numpy(np.array(range(0,batch,1)))
    
    #Shift the signal
    new_w = shift_operation(new_w.unsqueeze(0).repeat(batch,1,1,1).view(-1, sz, 1), shift, device).view(batch,3,sz,1)
    
    #Fit w into the range [0,1]. new_w is the same as ft
    #new_w = .5 * (torch.tanh(ootn) + 1)
    
    #stack the signal to fit the input size
    new_w = torch.stack([stack(ooti,sig_height,device) for ooti in new_w])
    
    #precision limit
    new_w = diff_round(new_w, precision_depth)
    
    #Convolution of ft and the shutter
    #gy = lay(new_w.unsqueeze(0).view([3,1,228,batch])).view([batch,3,224,1])
    gy = lay(new_w.transpose(0,3).transpose(0,1)).transpose(0,1).transpose(0,3)

    #Mask the signal to only affect the object
    if mask:
        gy_mask = torch.mul(gy,torch.transpose(mask,1,0))
    else:
        gy_mask = gy

    return (c + (1-c)*gy_mask), new_w, c

def upsample2d(img, size=224):
    upsample = nn.Upsample(size=size, mode='bilinear', align_corners=False)
    return upsample(torch.unsqueeze(img, 0))[0]

In [ ]:
signal_dir = "input_dump"

In [ ]:
signal_path = []
for subdir, dirs, files in os.walk(signal_dir):
    for filename in files:
        filepath = subdir + os.sep + filename
        
        if filepath.endswith(".json"):
            signal_path.append(filepath)
signal_configs = []
for sig in signal_path:
    with open(sig) as f:
        dd = json.load(f)
        dd['file_path'] = sig
        signal_configs.append(dd)

In [ ]:
signal_df = pd.DataFrame(signal_configs)

In [ ]:
signal_df[signal_df['classidx']=="456"]

In [ ]:
config = signal_df[signal_df['classidx']=="456"].to_dict('records')

In [ ]:
# Evaluating a signal
def load_signal(signal_path):
    return torch.load(signal_path.split(".")[0]+'.pt', map_location=device)[0]

def apply_signal(signal, sig_height, conv_size , batch_size, offset, original_images, apply_trans, ambient_light, targidx, classidx):
    target_acc, class_acc, max_acc = [], [], []
    gy, www, c = applyMask(signal, [offset, offset+batch_size], None, [0,0], sig_height, conv_size, 0)
    
    img_t2, img_b2, img_f2 = image_net_python.get_image_trip(original_images[0], original_images[1], device, apply_trans, ambient_light)
    inp2 = torch.pow(0.0000001 + torch.pow(img_b2,2.2) + gy*(torch.pow(img_f2,2.2)-torch.pow(img_t2,2.2)), 1/2.2)
    inp2 = torch.cat([forward_normalize(upsample2d(i,224)).unsqueeze(0) for i in inp2])
    inp2 = inp2.to(device)
    out2 = pretrained_model(inp2)
    prob2 = lay2(out2)
    maxcls2 = prob2.max(1)
    for i in range(batch_size):
        target_acc.append(prob2[i][targidx].item())
        class_acc.append(prob2[i][classidx].item())
        max_acc.append((maxcls2.indices[i].item(), prob2[i][maxcls2.indices[i].item()].item()))
    del img_t2
    del img_f2
    del inp2
    del out2
    del prob2
    del maxcls2
    torch.cuda.empty_cache()
    return target_acc, class_acc, max_acc
    

In [ ]:
batch_size = 32
def run_evaluation(config_dict):
    sig = load_signal(config_dict['file_path'])
    sig_height, conv_size = sig.size()[1], sig.size()[1]-251
    target_acc, class_acc, max_acc = [], [], []
    for i in range(0, int(sig_height/batch_size)*batch_size,batch_size):
        a,b,c = apply_signal(sig, sig_height, conv_size , batch_size, i, original_images, config_dict['apply_transformations']=="True", int(config_dict['ambient_light']), int(config_dict['targidx']), int(config_dict['classidx']))
        target_acc += a
        class_acc += b
        max_acc += c
    
    return target_acc, class_acc, max_acc

In [ ]:
original_images = [Image.open('images/original/col_amb_up.jpg'), Image.open('images/original/col_full_up.jpg')]
img_path = r'images/imagenet_classes/imagenet_images/imagenet_images/bow/384287867_37a8913d93_b.jpg'
img = Image.open(img_path)
img_full, img = TF.adjust_brightness(img, float(160/np.average(img))), TF.adjust_brightness(img, float(80/np.average(img)))
original_images = [img, img_full]


In [ ]:
imshow(original_images[1])

In [ ]:
target_acc, class_acc, max_acc = run_evaluation(config[0])

In [ ]:
avg_class_acc = np.array(class_acc).mean()
avg_target_acc = np.array(target_acc).mean()
dodge_success = np.array([1 if x[0] not in list(config[0]['classes_to_skip']) else 0 for x in max_acc]).mean()
target_success = np.array([1 if x[0]==int(config[0]['targidx']) else 0 for x in max_acc]).mean()
output_classes = {}
for cl in max_acc: 
    if cl[0] not in output_classes: output_classes[cl[0]]=0
    output_classes[cl[0]]+=1
print("Avg. accuracy for {} (original) - {}".format(class_dict[int(config[0]['classidx'])],avg_class_acc))
print("Avg. accuracy for {} (target)- {}".format(class_dict[int(config[0]['targidx'])],avg_target_acc))
print("Dodge success outside classes to skip - {}".format(dodge_success))
print("Target success - {}".format(target_success))
print("Output class histogram - ",output_classes)

In [ ]:
output_values = []
for conf in config:
    target_acc, class_acc, max_acc = run_evaluation(conf)
    
    avg_class_acc = np.array(class_acc).mean()
    avg_target_acc = np.array(target_acc).mean()
    dodge_success = np.array([1 if x[0] not in list(conf['classes_to_skip']) else 0 for x in max_acc])
    target_success = np.array([1 if x[0]==int(conf['targidx']) else 0 for x in max_acc])
    output_classes = {}
    for cl in max_acc: 
        if cl[0] not in output_classes: output_classes[cl[0]]=0
        output_classes[cl[0]]+=1
    print("Avg. accuracy for {} (original) - {}".format(class_dict[int(conf['classidx'])],avg_class_acc))
    print("Avg. accuracy for {} (target)- {}".format(class_dict[int(conf['targidx'])],avg_target_acc))
    print("Dodge success outside classes to skip - {}".format(dodge_success.mean()))
    print("Target success - {}".format(target_success.mean()))
    print("Output class histogram - ",output_classes)
    output_values.append((class_acc,target_acc,target_success,output_classes))

#output_values_config = ["Class acc", "Target acc", "Dodge(similar)","Attack Success"]
output_values_config = ["Class acc", "Target acc","Attack Success"]
input_configs = []
for t in config:
    input_configs.append(t['targidx']+"-"+str(class_dict[int(t['targidx'])][:10]))
    #input_configs.append(str(t)+"-adv")

df_list = []
for i,inpc in enumerate(input_configs):
    config_name = inpc
    for j in range(3):
        for k in range(len(output_values[i][j])):
            df_list.append([config_name, output_values[i][j][k], output_values_config[j]])
df_results = pd.DataFrame(df_list, columns = ['config','metric_value','metric_type'])

plot_title = 'Selected Classes; 10k; bow'
sns.barplot(data=df_results, x='config', y='metric_value',hue="metric_type").set_title(plot_title)
plt.xticks(rotation=30)
#plt.savefig('plots/'+plot_title+'.pdf')
